In [ ]:
! pip install selenium
! pip install pymysql

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
import pymysql

In [ ]:
# 결과 튜플 리스트
result = []

# 크롬으로 사이트 열기 (지역 검색까지)
driver = webdriver.Chrome()
driver.get("https://www.starbucks.co.kr/store/store_map.do")
driver.implicitly_wait(30)

sido_list = ["서울", "경기", "광주", "대구", "대전", "부산", "울산", "인천", "강원", "경남", "경북", "전남", "전북", "충남", "충북", "제주", "세종"]

# 시도별 점포 데이터 크롤링
for sido in sido_list:
    driver.find_element(By.LINK_TEXT, "지역 검색").click()
    driver.implicitly_wait(30)
    driver.find_element(By.LINK_TEXT, sido).click()
    driver.implicitly_wait(30)
    if sido != "세종":
        driver.find_element(By.LINK_TEXT, "전체").click()
        driver.execute_script("document.querySelector('#mCSB_3').style.cssText = 'overflow: visible;'")
        time.sleep(10)
        store_list = driver.find_elements(By.CSS_SELECTOR, "#mCSB_3_container > ul > li")
        for store in store_list:
            name = store.get_attribute("data-name")
            address_tel = store.find_element(By.TAG_NAME, "p").text.split("\n")
            address = address_tel[0]
            tel = address_tel[1]
            latitude = store.get_attribute("data-lat")
            longtitude = store.get_attribute("data-long")
            result.append((sido, name, address, tel, latitude, longtitude))
    else:
        driver.execute_script("document.querySelector('#mCSB_3').style.cssText = 'overflow: visible;'")
        time.sleep(10)
        store_list = driver.find_elements(By.CSS_SELECTOR, "#mCSB_3_container > ul > li")
        for store in store_list:
            name = store.get_attribute("data-name")
            address_tel = store.find_element(By.TAG_NAME, "p").text.split("\n")
            address = address_tel[0]
            tel = address_tel[1]
            latitude = store.get_attribute("data-lat")
            longtitude = store.get_attribute("data-long")
            result.append((sido, name, address, tel, latitude, longtitude))

    driver.quit()

In [ ]:
# 데이터프레임 csv 파일로 저장

df = pd.DataFrame(result, columns=["시도", "점포명", "주소", "전화번호", "위도", "경도"])
df.to_csv("../data/스타벅스 숙제.csv", encoding="utf8")

In [ ]:
# 결과물 데이터베이스에 저장
conn = pymysql.connect(
    host="localhost",
    user="jhm",
    password="1234",
    database="pydb",
    charset="utf8"
)
c = conn.cursor()
c.executemany(
    "INSERT INTO starbucks(sido, name, address, tel, latitude, longtitude) VALUES(%s, %s, %s, %s, %s, %s)",
    result
)
conn.commit()
conn.close()